In [1]:
import pandas as pd 

data = pd.read_csv("Corona_NLP_train.csv", encoding = 'latin1') 
data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [2]:
data = data.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis = 1)
data.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
import re
import preprocessor.api as p
from contractions import contractions_dict
from nltk.corpus import wordnet
from pattern.en import suggest
import wordninja
from numpy import nan
from nltk.tokenize import word_tokenize
import nltk
from stopword import stopwords
from nltk.corpus import words
from tqdm._tqdm_notebook import tqdm_notebook

def remove_nonASCII(text):
    #remove non english alphabet
    return re.sub(r'[^\x00-\x7f]',"", text)

def remove_repeated_characters(word):
    pattern = re.compile(r"(\w*)(\w)\2(\w*)")
    substitution_pattern = r"\1\2\3"
    while True:
        if wordnet.synsets(word):
            return word
        new_word = pattern.sub(substitution_pattern,word)
        if new_word != word:
            word = new_word
            continue
        else:
            return new_word

def expand_contractions(text):
    pattern = re.compile("({})".format("|".join(contractions_dict.keys())),flags = re.DOTALL| re.IGNORECASE)
    
    def replace_text(t):
        txt = t.group(0)
        if txt.lower() in contractions_dict.keys():
            return contractions_dict[txt.lower()]
        
    expand_text = pattern.sub(replace_text,text)
    return expand_text 

def spelling_checker(word):
    checker = suggest(word)
    return checker[0][0]

def remove_stopwords(w):
    text = []
    for word in w:
        if word not in stopwords:
              text.append(word)
    return text

def split_words(words):
    #split words which are not separated by space
    wlist = []
    for word in words:
        # 12 is decided based on distribution of word length in English
        if len(word)> 12:
            wlist = wlist + wordninja.split(word)
        else:
            wlist.append(word)
    return wlist

def preprocess_unit(text):
    text = remove_nonASCII(text)
    text = re.sub('\S+@\S*\s?', ' ', text) #remove email
    text = p.clean(text)    # remove URL, hashtag, @-mention, emojis
    text = re.sub(r'[^\w\s]', ' ', text) #remove punctuations
    text = re.sub("(\s\d+)","",text) #remove digits
    text = text.lower() #to lower case
    text = expand_contractions(text) #expand contraction
    Words = word_tokenize(text) #tokenization
    filtered_words = remove_stopwords(Words) #remove stopwords
    if len(filtered_words) > 0:
        """
        for word in filtered_words:
            if word in words.words():
                output.append(word)
            else:
                output.append(remove_repeated_characters(word))
        """
        #output = split_words(output)
        output = [spelling_checker(s) for s in filtered_words]
        print(output)
        return output
    else:
        return nan

#tqdm_notebook.pandas()
data['tokens'] = data['OriginalTweet'].progress_apply(preprocess_unit)
data.head()

['advice', 'talk', 'neighbours', 'family', 'exchange', 'phone', 'numbers', 'create', 'contact', 'list', 'phone', 'numbers', 'neighbours', 'schools', 'employer', 'chemist', 'up', 'set', 'online', 'shopping', 'accounts', 'pass', 'adequate', 'supplies', 'regular', 'beds', 'not', 'order']
['coronavirus', 'australia', 'woolworths', 'give', 'elderly', 'disabled', 'dedicated', 'shopping', 'hours', 'amid', 'could', 'outbreak']
['food', 'stock', 'not', 'one', 'empty', 'please', 'panic', 'enough', 'food', 'everyone', 'not', 'take', 'need', 'stay', 'calm', 'stay', 'safe']
['ready', 'go', 'superb', 'outbreak', 'not', 'paranoid', 'food', 'stock', 'literary', 'empty', 'serious', 'thing', 'please', 'panic', 'causes', 'shortage']
['news', 'regions', 'first', 'confirmed', 'could', 'case', 'came', 'sullivan', 'county', 'last', 'week', 'people', 'flocked', 'area', 'stores', 'purchase', 'cleaning', 'supplies', 'hand', 'sanitizer', 'food', 'toilet', 'paper', 'goods', 'reports']
['cashier', 'grocer', 'store

['stock', 'water', 'cause', 'utility', 'companies', 'shut', 'middle', 'anaemic', 'schools', 'close', 'their', 'doors', 'lose', 'work', 'cause', 'kid', 'no', 'go', 'can', 'afford', 'months', 'worth', 'food']
['dear', 'coronavirus', 'following', 'social', 'distending', 'rules', 'staying', 'home', 'prevent', 'spread', 'however', 'spent', 'alarming', 'amount', 'money', 'shopping', 'online', 'submit', 'expenses', 'reimbursement', 'let', 'know']
['global', 'food', 'prices', 'spread', 'could', 'intensified', 'across', 'several', 'biographies', 'could', 'see', 'downward', 'pressure', 'coming', 'months', 'due', 'continued', 'well', 'supplied', 'its', 'negative', 'impact', 'demand', 'resulting', 'virus']
['morning', 'everyone', 'great', 'safe', 'day']
['things', 'panic', 'buy', 'emergency', 'get', 'toilet', 'paper', 'important', 'afraid', 'worst', 'case', 'scenario', 'wash', 'tub', 'use', 'money', 'food', 'crazy']
['thank', 'grocer', 'clerk', 'went', 'grocer', 'store', 'today', 'looked', 'weary'

['followed', 'went', 'shopping', 'days', 'ago', 'pain', 'necessary', 'protect', 'grocer', 'shopping', 'consumer', 'reports']
['took', 'pictures', 'today', 'home', 'grocer', 'store', 'montgomery', 'county', 'md', 'no', 'flour', 'sugar', 'sweet', 'potatoes', 'potatoes', 'orange', 'juice', 'paper', 'towels', 'toilet', 'paper', 'low', 'meat', 'mac', 'amp', 'cheese']
['hate', 'grocer', 'shopping', 'general', 'swear', 'in', 'online', 'next', 'shop', 'not', 'deal', 'states', 'panic', 'buyers']
['rapid', 'delivery', 'food', 'order', 'made', 'since', 'no', 'shots', 'elsewhere', 'weeks', 'seemed', 'fine', 'email', 'listing', 'stock', 'deliver', 'one', 'bottle', 'orange', 'juice']
['thank', 'grocer', 'store', 'employees', 'working', 'hard', 'making', 'sure', 'everyone', 'getting', 'need', 'please', 'kind', 'not', 'fault', 'short', 'supplies']
['also', 'needs', 'said', 'going', 'grocer', 'store', 'not', 'done', 'completely', 'safely', 'meaning', 'without', 'risk', 'getting', 'spreading', 'reduce',

['government', 'says', 'start', 'social', 'distending', 'work', 'retail', 'not', 'talk', 'customers', 'store', 'll', 'for', 'going', 'catch', 'could']
['plus', 'side', 'could', 'saving', 'money', 'online', 'shopping', 'packages', 'china', 'hell', 'delayed', 'car', 'wan', 'na', 'use', 'public', 'transport', 'hell', 'wan', 'na', 'around', 'people', 'apparently', 'worked', 'wash', 'hands', 'wipe', 'ass']
['hearing', 'national', 'quarante', 'going', 'start', 'wednesday', 'th', 'said', 'stock', 'essentials', 'food', 'also', 'believe', 'going', 'get', 'ugly']
['food', 'banks', 'closing', 'source', 'enough', 'items', 'thanks', 'fckwits', 'panic', 'buying', 'everything', 'cares', 'broke', 'single', 'sum', 'three', 'kiss', 'feed', 'long', 'the', 'past', 'stockpiled', 'let', 'em', 'starve']
['dear', 'harder']
['first', 'shops', 'make', 'effort', 'could', 'really', 'win', 'or', 'game', 'make', 'real', 'difference']
['come', 'not', 'treading', 'real', 'people', 'creaked', 'stores', 'can', 'keep', 

['given', 'restrictions', 'around', 'could', 'fig', 'postponed', 'til', 'interest', 'food', 'security', 'increasing', 'amp', 'meeting', 'demand', 'thought', 'provoking', 'discussions', 'examining', 'food', 'issues', 'experienced', 'epidemic', 'amp', 'likely', 'experience', 'going', 'forward']
['people', 'not', 'taking', 'could', 'serious', 'get', 'stuck', 'indoors', 'no', 'food', 'did', 'want', 'necessary', 'stock', 'it', 'it', 'it', 'it']
['kept', 'hearing', 'on', 'na', 'travel', 'cheap', 'flights', 'infect', 'meanwhile', 'majority', 'borders', 'hundred', 'home', 'afraid', 'go', 'grocer', 'store']
['announces', 'loan', 'fund', 'administered', 'provide', 'financial', 'relief', 'k', 'based', 'business', 'including', 'nonprofits', 'it', 'full', 'amp', 'part', 'time', 'employees', 'impacted']
['past', 'past', 'sauce', 'penza', 'sold', 'grocer', 'store', 'everyone', 'dallas', 'become', 'italian', 'granddad']
['not', 'panic', 'not', 'fear', 'fear', 'mind', 'killerthe', 'press', 'working', '

...

['hey', 'likely', 'make', 'fortune', 'boom', 'online', 'shopping', 'due', 'going', 'ensure', 'pay', 'fair', 'up', 'tax', 'need', 'income', 'recovery', 'e', 'tax', 'avoidance', 'directive', 'useful', 'right']
['big', 'shout', 'superb', 'employees', 'working', 'long', 'hard', 'hours', 'dealing', 'grazed', 'general', 'public', 'salute', 'providing', 'us', 'thank']
['could', 'policy', 'idea', 'lawmakers', 'want', 'people', 'temporarily', 'stay', 'home', 'conduct', 'online', 'shopping', 'ri', 'temporarily', 'amp', 'e', 'sales', 'taxes', 'encourage', 'delivery', 'goods', 'amp', 'food', 'keep', 'w', 'lost', 'jobs']
['called', 'nice', 'toller', 'newton', 'baby', 'finds', 'hard', 'contain', 'emotion', 'describes', 'enemy', 'difficulty', 'finding', 'food', 'panic', 'buyers']
['not', 'limited', 'suppress', 'pharmacies', 'food', 'flies', 'shelves', 'others', 'survivalist', 'bent', 'mission', 'purchase', 'firearms']
['close', 'doors', 'employees', 'continue', 'paid', 'temporary', 'closure', 'online

['lord', 'please', 'protect', 'save', 'frontiers', 'healthcare', 'workers', 'po', 'bank', 'workers', 'superb', 'workers', 'please', 'save', 'could', 'please', 'sustain', 'let', 'feel', 'comfort', 'love']
['empty', 'many', 'stores', 'distributed', 'free', 'streets', 'up', 'prices', 'hand', 'sanitizers', 'went', 'limited', 'per', 'customer']
['experience', 'definitely', 'looks', 'like', 'evening', 'superb', 'routine', 'see', 'even', 'taken']
['people', 'crowding', 'doors', 'superb', 'saint', 'denis', 'day', 'announced', 'tightening', 'travel', 'restrictions']
['literally', 'every', 'single', 'grocer', 'store', 'retail', 'worker', 'coming', 'home', 'work', 'freak']
['consumer', 'reporting', 'agree', 'interest', 'pray', 'could', 'stories', 'however', 'crucial', 'time', 'active', 'incredible', 'journalism', 'things', 'dont', 'happen', 'shadows', 'away', 'public', 'view', 'keep', 'fighting']
['cologne', 'germany', 'applauding', 'health', 'workers', 'retail', 'store', 'employees', 'truck', 'd

['politicians', 'need', 'come', 'retail', 'store', 'see', 'people', 'not', 'practicing', 'self', 'quarante', 'social', 'distending', 'not', 'even', 'close', 'elected', 'make', 'hard', 'soon', 'obviously', 'people', 'not', 'making']
['your', 'one', 'people', 'excessively', 'stocked', 'food', 'could', 'panic', 'please', 'least', 'donate', 'can', 'wont', 'use', 'shelters', 'food', 'banks', 'instead', 'letting', 'go', 'waste']
['cuba', 'becomes', 'cheaper', 'live']
['win', 'baby', 'please', 'choose', 'another', 'brand']
['canadians', 'not', 'need', 'panic', 'food', 'shortage', 'amid', 'could', 'experts', 'say']
['please', 'consider', 'shopping', 'one', 'faces', 'collection', 'online', 'please', 'between']
['stop', 'panic', 'buying', 'consider', 'mass', 'gathering', 'well', 'sprang', 'berpusu', 'pus', 'decatur', 'dear', 'grocer', 'store', 'tu']
['possible', 'get', 'virus', 'given', 'cashier', 'superb', 'asking', 'real', 'question']
['prime', 'minister', 'austin', 'trudeau', 'asks', 'canadia

['thursday', 'grocer', 'store', 'ulcer', 'three', 'ply', 'toilet', 'tissue', 'sure', 'line']
['home', 'need', 'groceriesdidnt', 'stock', 'weeks', 'buy', 'family', 'dogs', 'amp', 'case', 'need', 'food', 'fruits', 'veggies', 'chicken', 'ground', 'beef', 'potatoes', 'bread', 'soup', 'past']
['protect', 'food', 'workers', 'may', 'find', 'real', 'shortage', 'soon', 'demand', 'food', 'workers', 'get', 'full', 'pay', 'suffer', 'no', 'penalties', 'related', 'illness', 'not', 'member', 'union', 'join', 'us', 'organise', 'send', 'clear', 'message', 'employers', 'not', 'abused']
['saw', 'elderly', 'woman', 'walk', 'grocer', 'store', 'live', 'across', 'street', 'every', 'time', 'see', 'someone', 'high', 'risk', 'catching', 'like']
['self', 'isolation', 'symptoms', 'virus', 'fine', 'try', 'shopping', 'online', 'major', 'suppress', 'earliest', 'delivery', 'weeks', 'unless', 'great', 'efforts', 'made', 'improve', 'virus', 'forced', 'visit', 'shops']
['wife', 'got', 'grocer', 'union', 'st', 'danger', 

['ministry', 'fixed', 'food', 'prices', 'order', 'prevent', 'its', 'hiding', 'outbreak']
['please', 'fix', 'food', 'prices', 'essentials', 'immediate', 'effect', 'signs', 'collision', 'retail', 'shops', 'basic', 'items', 'hand', 'sanitizers', 'hand', 'soap', 'mineral', 'water']
['believe', 'said', 'please', 'respectful', 'nice', 'anyone', 'working', 'time', 'specifically', 'grocer', 'store', 'employees', 'dry', 'nurses', 'er', 'registers', 'etc']
['people', 'ages', 'support', 'older', 'adults', 'could', 'period', 'know', 'meditations', 'loved', 'one', 'taking', 'extra', 'hand', 'monitor', 'food', 'amp', 'medical', 'supplies', 'stock', 'non', 'perishable', 'food', 'items']
['une', 'certain', 'de', 'personnel', 'se', 'present', 'aux', 'ports', 'dun', 'superb', 'saint', 'denis', 'en', 'banlieue', 'parisienne', 'au', 'lendemain', 'de', 'announce', 'du', 'president', 'matron', 'renforant', 'les', 'restrictions', 'de', 'placement', 'face', 'au']
['countries', 'affected', 'try', 'control', 's